Find cancelation percentage on each day made by both client and driver and filter for not banned users

In [1]:
%CREATE potato.db 

In [2]:
Create table  Trips (id int, client_id int, driver_id int, city_id int, status varchar(50), request_at varchar(50));

In [3]:
Create table Users (users_id int, banned varchar(50), role varchar(50));

In [4]:
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'completed', '2013-10-01'),
('2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'),
('3', '3', '12', '6', 'completed', '2013-10-01'),
('4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'),
('5', '1', '10', '1', 'completed', '2013-10-02'),
('6', '2', '11', '6', 'completed', '2013-10-02'),
('7', '3', '12', '6', 'completed', '2013-10-02'),
('8', '2', '12', '12', 'completed', '2013-10-03'),
('9', '3', '10', '12', 'completed', '2013-10-03'),
('10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03');

In [5]:
insert into Users (users_id, banned, role) values ('1', 'No', 'client'),
('2', 'Yes', 'client'),
('3', 'No', 'client'),
('4', 'No', 'client'),
('10', 'No', 'driver'),
('11', 'No', 'driver'),
('12', 'No', 'driver'),
('13', 'No', 'driver');

In [6]:
select * from Users

users_id,banned,role
1,No,client
2,Yes,client
3,No,client
4,No,client
10,No,driver
11,No,driver
12,No,driver
13,No,driver


In [7]:
select * from Trips

id,client_id,driver_id,city_id,status,request_at
1,1,10,1,completed,2013-10-01
2,2,11,1,cancelled_by_driver,2013-10-01
3,3,12,6,completed,2013-10-01
4,4,13,6,cancelled_by_client,2013-10-01
5,1,10,1,completed,2013-10-02
6,2,11,6,completed,2013-10-02
7,3,12,6,completed,2013-10-02
8,2,12,12,completed,2013-10-03
9,3,10,12,completed,2013-10-03
10,4,13,12,cancelled_by_driver,2013-10-03


In [8]:
with cte as (select t.request_at, sum(case when status in ('cancelled_by_client','cancelled_by_driver') then 1 else 0 end) as count_cancelation,
    count(1) as num_of_requests from trips t
inner join users u on t.client_id = u.users_id
inner join users u1 on t.driver_id = u1.users_id

where u.banned = 'No' and u1.banned = 'No'
group by t.request_at)

select *, ( 1.0 *count_cancelation/num_of_requests)*100 as cancelation_percentage from cte

request_at,count_cancelation,num_of_requests,cancelation_percentage
2013-10-01,1,3,33.3333333333333
2013-10-02,0,2,0.0
2013-10-03,1,2,50.0
